In [1]:
import os
os.chdir('/mnt/ufs18/home-230/pandavis/cellular_resilience/CellResilienceModel/codebase')


import re
import random
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna

from src.linear_regression import ElasticLinear
from src.train import train
from src.mlp import MLP
from src.evaluate import evaluate

import torch
from torch.utils.data import TensorDataset, DataLoader
import lightning as L
import pickle

seed = 42
random.seed(seed)
np.random.seed(seed)

/mnt/home/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/mnt/ufs18/home-230/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
banksy_matrix = sc.read_h5ad('../data/BreastCancer10xGenomics_Rep1/exported_data/banksy_matrix.h5ad')
banksy_matrix

AnnData object with n_obs × n_vars = 156831 × 900
    obs: 'celltype', 'x_centroid', 'y_centroid', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_10_genes', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'is_nbr', 'k'

In [3]:
pd.set_option('display.max_rows', None)
print(banksy_matrix.to_df().columns[0:300])
print(banksy_matrix.to_df().columns[300:600])
print(banksy_matrix.to_df().columns[600:])

Index(['ABCC11', 'ACTA2', 'ACTG2', 'ADAM9', 'ADGRE5', 'ADH1B', 'ADIPOQ',
       'AGR3', 'AHSP', 'AIF1',
       ...
       'TUBB2B', 'TYROBP', 'UCP1', 'USP53', 'VOPP1', 'VWF', 'WARS', 'ZEB1',
       'ZEB2', 'ZNF562'],
      dtype='object', length=300)
Index(['ABCC11_nbr_0', 'ACTA2_nbr_0', 'ACTG2_nbr_0', 'ADAM9_nbr_0',
       'ADGRE5_nbr_0', 'ADH1B_nbr_0', 'ADIPOQ_nbr_0', 'AGR3_nbr_0',
       'AHSP_nbr_0', 'AIF1_nbr_0',
       ...
       'TUBB2B_nbr_0', 'TYROBP_nbr_0', 'UCP1_nbr_0', 'USP53_nbr_0',
       'VOPP1_nbr_0', 'VWF_nbr_0', 'WARS_nbr_0', 'ZEB1_nbr_0', 'ZEB2_nbr_0',
       'ZNF562_nbr_0'],
      dtype='object', length=300)
Index(['ABCC11_nbr_1', 'ACTA2_nbr_1', 'ACTG2_nbr_1', 'ADAM9_nbr_1',
       'ADGRE5_nbr_1', 'ADH1B_nbr_1', 'ADIPOQ_nbr_1', 'AGR3_nbr_1',
       'AHSP_nbr_1', 'AIF1_nbr_1',
       ...
       'TUBB2B_nbr_1', 'TYROBP_nbr_1', 'UCP1_nbr_1', 'USP53_nbr_1',
       'VOPP1_nbr_1', 'VWF_nbr_1', 'WARS_nbr_1', 'ZEB1_nbr_1', 'ZEB2_nbr_1',
       'ZNF562_nbr_1'],
      dtype='

In [4]:
banksy_matrix.obs['celltype'].value_counts()

celltype
Luminal epithelial cells      61326
T cells                       28479
Fibroblasts                   20672
Macrophages                   14007
Endothelial cells             12276
B cells                        9709
Myoepithelial cells            9394
Cancer cells                    968
Name: count, dtype: int64

In [5]:
celltype_adata = banksy_matrix[banksy_matrix.obs['celltype'] == 'Luminal epithelial cells  ']
celltype_adata

View of AnnData object with n_obs × n_vars = 61326 × 900
    obs: 'celltype', 'x_centroid', 'y_centroid', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_10_genes', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'is_nbr', 'k'

In [6]:
n = len(banksy_matrix.var) // 3

X = celltype_adata.X[:, n:2*n]
G = celltype_adata.X[:, :n]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, G, test_size=0.2, random_state=42
)

In [8]:
results = train(
    ElasticLinear,
    X_train,
    y_train,
    X_test,
    y_test,
    loss_fn=torch.nn.MSELoss(),
    learning_rate=0.05,
    max_epochs=100,
    batch_size=None,
    #early_stopping=True,  
   # patience=10,
)

INFO: Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO:lightning.pytorch.utilities.rank_zero:Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
/mnt/home/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/home/pandavis/anaconda3/envs/pytorch/lib/python ...
INFO: GPU available: True (cuda)

Training: |                                                                                                   …

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [8]:
#evaluate(results['model'], X_test, y_test)

NameError: name 'results' is not defined

In [15]:
cell_types = [
    "Luminal epithelial cells  ",
    "T cells  ",
    "Fibroblasts  ",
    "Macrophages  ",
    "Endothelial cells  ",
    "B cells  ",
    "Myoepithelial cells  ",
]

weights = {}

for cell_type in cell_types:
    print(f"Processing {cell_type}...")

    celltype_adata = banksy_matrix[banksy_matrix.obs['celltype'] == cell_type]

    #n = len(banksy_matrix.var) // 3
    X = celltype_adata.X[:, n:2*n]
    G = celltype_adata.X[:, :n]

    X_train, X_test, y_train, y_test = train_test_split(X, G, test_size=0.2, random_state=42)

    # Train ElasticLinear model
    results = train(
        model_class=ElasticLinear,
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        y_test=y_test,
        loss_fn=torch.nn.MSELoss(),
        learning_rate=0.05,
        max_epochs=100,
        l1_lambda=0.0,
        l2_lambda=0.1,
    )

    metrics = evaluate(
        model=results['model'], 
        X_test=X_test, 
        y_test=y_test
    )
    
    weights[cell_type] = results['weights']
    
    #np.save(f"results/{cell_type}_elastic_net_weights.npy", results["weights"])
    print(f"ElasticNet Metrics for {cell_type}: {metrics}")

Processing Luminal epithelial cells  ...


INFO: Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO:lightning.pytorch.utilities.rank_zero:Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
/mnt/home/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/home/pandavis/anaconda3/envs/pytorch/lib/python ...
INFO: GPU available: True (cuda)

Training: |                                                                                                   …

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


ElasticNet Metrics for Luminal epithelial cells  : {'RMSE': 0.5443497, 'R2': 0.0033676581882638957}
Processing T cells  ...


...

ElasticNet Metrics for B cells  : {'RMSE': 1.0923301, 'R2': 0.006627643754308957}
Processing Myoepithelial cells  ...


/mnt/home/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.
/mnt/home/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                                                                                                   …

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


ElasticNet Metrics for Myoepithelial cells  : {'RMSE': 0.9224537, 'R2': 0.011596127755965803}


array([-3.6700308e-01,  7.3091060e-01,  9.0775937e-01, ...,
       -2.1476422e-04,  8.5813976e-05,  5.6495512e-04], dtype=float32)

In [16]:
fro_norms = {}

for cell_type in cell_types:

    weight_celltype = weights[cell_type]
    W = weight_celltype[300:].reshape(300, 300)

    fro_norm = np.linalg.norm(W, ord='fro')
    fro_norms[cell_type] = fro_norm

In [17]:
fro_norms

{'Luminal epithelial cells  ': 0.114897914,
 'T cells  ': 0.18012649,
 'Fibroblasts  ': 0.18268244,
 'Macrophages  ': 0.23536664,
 'Endothelial cells  ': 0.26338494,
 'B cells  ': 0.28131753,
 'Myoepithelial cells  ': 0.3918243}